In [27]:
INPUTDIR = "./"

In [28]:
sampled_paperid = set()
with open(INPUTDIR + "sampled_paperid.txt", "r") as f:
    for line in f.readlines():
        paperid = line.rstrip().split("\t")[0]
        sampled_paperid.add(paperid)

In [29]:
paperid2hindex = {}
hindex2paperid = {}
hedge2node = []
hedge2nodepos =[]
authorid2vindex = {}
vindex2authorid = {}
numhedges = 0
numnodes = 0

In [30]:
with open(INPUTDIR + "hypergraph.txt", "r") as f, open(INPUTDIR + "hypergraph_pos.txt", "r") as pf:
    for line, pline in zip(f.readlines(), pf.readlines()):
        nodes = line.rstrip().split("\t")
        node_poses = pline.rstrip().split("\t")[1:]
        paperid = nodes[0]
        nodes = nodes[1:]
        if len(nodes) == 1:
            continue
        elif paperid not in sampled_paperid:
            continue
        hindex = numhedges
        paperid2hindex[paperid] = hindex
        hindex2paperid[hindex] = paperid
        numhedges += 1
        hedge2node.append([])
        hedge2nodepos.append([])
        
        for authorid, _vpos in zip(nodes, node_poses):
            if authorid not in authorid2vindex:
                vindex = numnodes
                authorid2vindex[authorid] = vindex
                vindex2authorid[vindex] = authorid
                numnodes += 1
            vindex = authorid2vindex[authorid]
            if int(_vpos) == 1:
                vpos = 0
            elif int(_vpos) == len(nodes):
                vpos = 2
            else:
                vpos = 1
            hedge2node[hindex].append(int(vindex))
            hedge2nodepos[hindex].append(int(vpos))

In [31]:
OUTPUTDIR = "/workspace/WithInHyperedgeNodeLabel/downstreamdata/AMiner/"

In [32]:
with open(OUTPUTDIR + "hypergraph.txt", "w") as f:
    for hindex in range(numhedges):
        paperid = "'" + hindex2paperid[hindex] + "'"
        nodes = [str(v) for v in hedge2node[hindex]]
        line = "\t".join([paperid] + nodes)
        f.write(line + "\n")
                         
with open(OUTPUTDIR + "hypergraph_pos.txt", "w") as f:
    for hindex in range(numhedges):
        paperid = "'" + hindex2paperid[hindex] + "'"
        nodeposes = [str(vpos) for vpos in hedge2nodepos[hindex]]
        line = "\t".join([paperid] + nodeposes)
        f.write(line + "\n")

In [ ]:
from sklearn.model_selection import train_test_split

In [36]:
sampled_hindex = set()
with open(INPUTDIR + "sampled_paperid_10000.txt", "r") as f:
    for line in f.readlines():
        paperid = line.rstrip()
        if paperid in paperid2hindex:
            hindex = paperid2hindex[paperid]
            sampled_hindex.add(hindex)

In [37]:
print(len(sampled_hindex))

10000


In [38]:
with open(OUTPUTDIR + "sampled_hset_10000.txt", "w") as f:
    for hindex in sampled_hindex:
        f.write(str(hindex) + "\n")

In [41]:
_train_hindex, test_hindex = train_test_split(list(sampled_hindex), test_size=0.2, random_state=21)
train_hindex, valid_hindex = train_test_split(_train_hindex, test_size=0.25, random_state=21)

In [42]:
print(len(valid_hindex))
print(len(test_hindex))

2000
2000


In [43]:
with open(OUTPUTDIR + "valid_hindex_10000.txt", "w") as f:
    for hindex in valid_hindex:
        paperid = hindex2paperid[hindex]
        f.write(paperid + "\n")

with open(OUTPUTDIR + "test_hindex_10000.txt", "w") as f:
    for hindex in test_hindex:
        paperid = hindex2paperid[hindex]
        f.write(paperid + "\n")